In [ ]:
import os
import csv
import json
import joblib
import numpy as np
from typing import List, Dict, Tuple
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.model_selection import train_test_split
random_state =42

vocab_size = 3000 

RAW_CSV = "data/raw/sms.csv"              # file gốc: 2 cột "label","text"
TRAIN_OUT = "data/processed/train.csv"    # output train
TEST_OUT  = "data/processed/test.csv"     # output test
VEC_PKL   = "artifacts/vectorizer.pkl"    # pickle vectorizer cho Bước 3/4
VOCAB_TXT = "artifacts/vocab.txt"         # vocab (tham khảo)
TEST_SIZE = 0.2                           # 80/20 split

# -----------------------------
# 1) STOPWORDS & CLEANING
# -----------------------------
STOPWORDS = {
    "a","an","the","is","are","am","was","were","be","been","being","i","you","he","she","it","we","they","me","him","her","us","them",
    "this","that","these","those","there","here","of","to","in","on","for","from","with","by","at","as","about","into","over","after",
    "before","between","and","or","but","if","then","so","because","while","than","though","although","not","no","do","does","did","doing",
    "done","dont","didnt","doesnt","isnt","arent","wasnt","werent","cant","cannot","my","your","his","her","its","our","their",
    "have","has","had","having","will","would","shall","should","can","could","may","might","must",
    "im","ive","youre","hes","shes","weve","theyre","ill","youll","dont","cant","wont","didnt","couldnt","shouldnt","wouldnt","lets"
}

def keep_letters_and_spaces(s: str) -> str:
    """
    Chỉ giữ lại chữ cái (a-z) và khoảng trắng
    
    Tham số:
        s: chuỗi đã lowercase
        
    Trả về:
        Chuỗi chỉ chứa a-z và space
        
    Ví dụ:
        "hello123!@#world" -> "hello   world"
    """
    out = []
    for ch in s:
        if 'a' <= ch <= 'z' or ch == ' ':
            out.append(ch)
        else:
            out.append(' ')
    return ''.join(out)


def clean_text(text: str, stopwords: set) -> str: #Làm sạch văn bản: lowercase -> loại ký tự đặc biệt -> loại stopword
    """
        text: văn bản cần làm sạch
        stopwords: tập hợp các từ dừng
        
    Trả về:
        Văn bản đã làm sạch
        
    Các bước:
        1. Chuyển về chữ thường
        2. Loại bỏ ký tự đặc biệt, chỉ giữ a-z và space
        3. Loại bỏ khoảng trắng thừa
        4. Loại bỏ stopwords
        5. Loại bỏ từ có độ dài <= 1
    """
    text = text.lower()# Bước 1: lowercase
    

    text = keep_letters_and_spaces(text)# Bước 2: chỉ giữ chữ cái và space
    
    text = ' '.join(text.split()) # Bước 3: loại khoảng trắng thừa
    
    
    words = text.split()
    words = [w for w in words if w not in stopwords and len(w) > 1]# Bước 4 & 5: tách từ, loại stopwords và từ ngắn
    
    return ' '.join(words)


# -----------------------------
# *) ĐỌC VÀ LÀM SẠCH DỮ LIỆU
# -----------------------------
def load_and_clean_data(csv_path: str, stopwords: set) -> Tuple[List[str], List[int]]:
    """
    Đọc file CSV và làm sạch dữ liệu
    
    Tham số:
        csv_path: đường dẫn file CSV (cột 1: label, cột 2: text)
        stopwords: tập stopwords
        
    Trả về:
        (texts, labels) - danh sách văn bản đã làm sạch và nhãn (0=ham, 1=spam)
    """
    print(f"\n📖 Đọc dữ liệu từ: {csv_path}")
    
    texts = []
    labels = []
    
    # Đọc file CSV
    with open(csv_path, 'r', encoding='latin-1') as f:
        reader = csv.reader(f)
        header = next(reader)  # bỏ qua header
        
        for row in reader:
            if len(row) < 2:
                continue
                
            label_str = row[0].strip().lower()  # 'ham' hoặc 'spam'
            raw_text = row[1].strip()
            
            
            cleaned = clean_text(raw_text, stopwords) # Làm sạch văn bản
            
            
            label_int = 1 if label_str == 'spam' else 0 # Chuyển label thành số: ham=0, spam=1
            
                             
            if cleaned:# Chỉ giữ lại nếu văn bản không rỗng sau khi làm sạch
                texts.append(cleaned)
                labels.append(label_int)
    
    print(f"✅ Đọc thành công {len(texts)} tin nhắn")
    print(f"   - HAM (0): {labels.count(0)} tin")
    print(f"   - SPAM (1): {labels.count(1)} tin")
    
    return texts, labels


# -----------------------------
# 3) CHIA TRAIN/TEST
# -----------------------------
def split_train_test(texts: List[str], labels: List[int], 
                     test_size: float = 0.2, 
                     random_state: int = 42) -> Tuple:
    """
    Chia dữ liệu thành tập train và test
    
    Tham số:
        texts: danh sách văn bản
        labels: danh sách nhãn
        test_size: tỷ lệ test (0.2 = 20%)
        random_state: seed cho random
        
    Trả về:
        X_train, X_test, y_train, y_test
    """
    print(f"\n Chia dữ liệu: {int((1-test_size)*100)}% train, {int(test_size*100)}% test")
    
    X_train, X_test, y_train, y_test = train_test_split(
        texts, labels, 
        test_size=test_size, 
        random_state=random_state,
        stratify=labels  # đảm bảo tỷ lệ ham/spam đều trong train và test
    )
    
    print(f"✅ Train: {len(X_train)} mẫu")
    print(f"✅ Test:  {len(X_test)} mẫu")
    
    return X_train, X_test, y_train, y_test


# -----------------------------
# 4) VECTOR HÓA VĂN BẢN
# -----------------------------
def create_vectorizer(vocab_size: int = 3000, method: str = 'tfidf'):
    """
    Tạo vectorizer để chuyển văn bản thành vector số
    
    Tham số:
        vocab_size: số lượng từ trong vocabulary
        method: 'tfidf' hoặc 'count'
        
    Trả về:
        Vectorizer object
        
    Giải thích:
        - TF-IDF: Term Frequency - Inverse Document Frequency
          + Đo lường mức độ quan trọng của từ trong văn bản
          + Từ xuất hiện nhiều trong 1 văn bản nhưng ít trong toàn bộ → quan trọng
          
        - Count: Đơn giản đếm số lần xuất hiện của từ
    """
    print(f"\n🔢 Tạo vectorizer ({method.upper()}) với vocab_size={vocab_size}")
    
    if method == 'tfidf':
        vectorizer = TfidfVectorizer(
            max_features=vocab_size,  # giới hạn số từ
            ngram_range=(1, 2),       # unigram và bigram (từ đơn và cụm 2 từ)
            min_df=2,                 # từ phải xuất hiện ít nhất 2 lần
            max_df=0.95               # loại từ xuất hiện quá nhiều (>95% văn bản)
        )
    else:
        vectorizer = CountVectorizer(
            max_features=vocab_size,
            ngram_range=(1, 2),
            min_df=2,
            max_df=0.95
        )
    
    return vectorizer


def fit_and_transform(vectorizer, X_train: List[str], X_test: List[str]):
    """
    Fit vectorizer trên train và transform cả train lẫn test
    
    Tham số:
        vectorizer: TfidfVectorizer hoặc CountVectorizer
        X_train: danh sách văn bản train
        X_test: danh sách văn bản test
        
    Trả về:
        X_train_vec, X_test_vec (dạng sparse matrix)
        
    Lưu ý:
        - Fit chỉ trên train để tránh data leakage
        - Transform cả train và test bằng cùng vocabulary
    """
    print("⚙️ Đang fit vectorizer trên tập train...")
    X_train_vec = vectorizer.fit_transform(X_train)
    
    print("⚙️ Đang transform tập test...")
    X_test_vec = vectorizer.transform(X_test)
    
    print(f"✅ Train vector shape: {X_train_vec.shape}")
    print(f"✅ Test vector shape:  {X_test_vec.shape}")
    print(f"📚 Vocabulary size: {len(vectorizer.vocabulary_)}")
    
    return X_train_vec, X_test_vec


# -----------------------------
# 5) LƯU DỮ LIỆU
# -----------------------------
def save_processed_data(X_train, y_train, X_test, y_test, 
                       train_out: str, test_out: str):
    """
    Lưu dữ liệu đã xử lý ra file CSV
    
    Tham số:
        X_train, y_train: dữ liệu train (văn bản và nhãn)
        X_test, y_test: dữ liệu test
        train_out: đường dẫn file train output
        test_out: đường dẫn file test output
    """
    print(f"\n Lưu dữ liệu đã xử lý...")
    
    # Tạo thư mục nếu chưa có
    os.makedirs(os.path.dirname(train_out), exist_ok=True)
    
    # Lưu train
    train_df = pd.DataFrame({
        'text': X_train,
        'label': y_train
    })
    train_df.to_csv(train_out, index=False, encoding='utf-8')
    print(f"✅ Đã lưu train: {train_out}")
    
    # Lưu test
    test_df = pd.DataFrame({
        'text': X_test,
        'label': y_test
    })
    test_df.to_csv(test_out, index=False, encoding='utf-8')
    print(f"✅ Đã lưu test: {test_out}")


def save_vectorizer_and_vocab(vectorizer, vec_pkl: str, vocab_txt: str):
    """
    Lưu vectorizer và vocabulary
    
    Tham số:
        vectorizer: đối tượng vectorizer đã fit
        vec_pkl: đường dẫn lưu vectorizer (pickle)
        vocab_txt: đường dẫn lưu vocabulary (text)
    """
    print(f"\n💾 Lưu vectorizer và vocabulary...")
    
    # Tạo thư mục
    os.makedirs(os.path.dirname(vec_pkl), exist_ok=True)
    
    # Lưu vectorizer
    joblib.dump(vectorizer, vec_pkl)
    print(f"✅ Đã lưu vectorizer: {vec_pkl}")
    
    # Lưu vocabulary
    vocab = vectorizer.get_feature_names_out()
    with open(vocab_txt, 'w', encoding='utf-8') as f:
        for word in vocab:
            f.write(word + '\n')
    print(f"✅ Đã lưu vocabulary ({len(vocab)} từ): {vocab_txt}")


# -----------------------------
# 6) PIPELINE CHÍNH
# -----------------------------
def main():
    """
    Hàm chính: chạy toàn bộ pipeline
    """
    print("="*70)
    print("PIPELINE TIỀN XỬ LÝ DỮ LIỆU SMS SPAM".center(70))
    print("="*70)
    
    # Set random seed
    random.seed(RANDOM_STATE)
    np.random.seed(RANDOM_STATE)
    
    # Bước 1: Đọc và làm sạch dữ liệu
    texts, labels = load_and_clean_data(RAW_CSV, STOPWORDS)
    
    # Bước 2: Chia train/test
    X_train, X_test, y_train, y_test = split_train_test(
        texts, labels, 
        test_size=TEST_SIZE, 
        random_state=RANDOM_STATE
    )
    
    
    save_processed_data(X_train, y_train, X_test, y_test, TRAIN_OUT, TEST_OUT) # Bước 3: Lưu dữ liệu đã làm sạch
    
   
    vectorizer = create_vectorizer(vocab_size=VOCAB_SIZE, method='tfidf') # Bước 4: Tạo và fit vectorizer
    X_train_vec, X_test_vec = fit_and_transform(vectorizer, X_train, X_test)
    
   
    save_vectorizer_and_vocab(vectorizer, VEC_PKL, VOCAB_TXT) # Bước 5: Lưu vectorizer và vocabulary
    
    # Thống kê cuối cùng
    print("\n" + "="*70)
    print(" TỔNG KẾT".center(70))
    print("="*70)
    print(f"✅ Tổng số mẫu: {len(texts)}")
    print(f"✅ Train set: {len(X_train)} mẫu")
    print(f"✅ Test set: {len(X_test)} mẫu")
    print(f"✅ Vocabulary size: {len(vectorizer.vocabulary_)}")
    print(f"✅ Vector shape: ({len(X_train)}, {len(vectorizer.vocabulary_)})")
    print("\n📁 Files đã tạo:")
    print(f"   - {TRAIN_OUT}")
    print(f"   - {TEST_OUT}")
    print(f"   - {VEC_PKL}")
    print(f"   - {VOCAB_TXT}")
    print("\n✨ Pipeline hoàn thành!")
    print("="*70)
    
    return X_train_vec, X_test_vec, y_train, y_test, vectorizer



: 

In [ ]:
index = 0
visited[start] = False
while (visited[end] == False):
    for x in range 

In [ ]:
ds = []
item1 = tuple([1,2]); item2 = tuple([13,2])
ds.append(item1)
ds.append(item2)
visited =[]
for _ in range(n):
    